In [2]:
!pip install tensorflow-text

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d aminesedkaoui/epinions-ratings-500k

In [37]:
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import random
from keras import datasets, layers, models

In [ ]:
!unzip "/content/epinions-ratings-500k.zip" -d "/content/"

In [ ]:
df = pd.read_csv('/content/epinions_rating_500k.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

In [3]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from lfoat to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item,df_user_item

In [7]:

users = df['User_ID'].unique()
items = df['Item_ID'].unique()
df_train = df.copy()

New_User_IDs = random.sample(range(10000,50000),df['User_ID'].nunique())
New_Item_IDs = random.sample(range(100000,200000),df['Item_ID'].nunique())
i = 0
for d in users :
   df_train['User_ID'].replace({d : New_User_IDs[i]}, inplace=True)
   i+=1

i = 0
for d in items :
   df_train['Item_ID'].replace({d : New_Item_IDs[i]}, inplace=True)
   i+=1

In [8]:
df_train['one']=df_train['rating'].apply(lambda x: 1 if x==1 else 0)
df_train['two']=df_train['rating'].apply(lambda x: 1 if x==2 else 0)
df_train['three']=df_train['rating'].apply(lambda x: 1 if x==3 else 0)
df_train['four']=df_train['rating'].apply(lambda x: 1 if x==4 else 0)
df_train['five']=df_train['rating'].apply(lambda x: 1 if x==5 else 0)
df_train['six']=df_train['rating'].apply(lambda x: 1 if x==6 else 0)

In [23]:
#GMF

def Create_GMF(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(6, activation='softmax', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF

from keras import backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

def Train_GMF(model,X_train,Y_train,nbrEpochs):
  model.compile(optimizer = 'adam',
                    loss = tf.keras.losses.categorical_crossentropy ,
                    metrics=['accuracy'])
  model.fit(X_train,Y_train,epochs = nbrEpochs)
  return model

def Fill_Cf_Matrix(model,dataframe,userIDs,itemIDs):
  for user in userIDs:
    for item in itemIDs:
      if dataframe.loc[user][item] == 0:
        dataframe.loc[user][item] = model.predict([user,item],verbose=1)
  
  return dataframe


In [10]:
GMF = Create_GMF(df_train['User_ID'].max() + 1,df_train['Item_ID'].max() + 1,64)

In [24]:
from sklearn.model_selection import train_test_split
df_train_sample = df_train[:2000]
X_train, X_test,y_train,y_test= train_test_split(df_train_sample[['User_ID','Item_ID']],df_train_sample[['one','two','three','four','five','six']],stratify=df_train_sample[['one','two','three','four','five','six']])
#X_train, X_test,y_train,y_test= train_test_split(df_train[['User_ID','Item_ID']],df_train['rating'],stratify=df_train['rating'])

In [ ]:
X_test

In [ ]:
GMF_trained = Train_GMF(GMF,[X_train['User_ID'],X_train['Item_ID']],y_train,1)

In [13]:
user_item_matrix , df_mat = DatasetToUserItemDataFrame(df_train_sample,'User_ID','Item_ID','rating')

In [ ]:
df_mat.head()

In [ ]:
X_test.shape

In [35]:
GMF_trained.evaluate([X_test['User_ID'],X_test['Item_ID']],y_test)

16/16 [==============================] - 0s 3ms/step - loss: 1.2933 - accuracy: 0.8380


[1.2932934761047363, 0.8379999995231628]

In [ ]:
GMF_trained.predict([X_test['User_ID'],X_test['Item_ID']])

In [ ]:
df_mat_filled = Fill_Cf_Matrix(GMF_trained,df_mat,New_User_IDs,New_Item_IDs)

In [ ]:
#Autoencoder

encoder_input = layers.Input(shape=(X_train.shape[1]),name='user_item')
flat = layers.Flatten()(encoder_input)
hid_encoder = layers.Dense(512,activation="relu")(flat)
encoder_output = layers.Dense(256,activation="relu")(hid_encoder)

decoder_input = layers.Dense(512,activation="relu")(encoder_output)
decoder_output = layers.Dense(X_train.shape[1],activation="relu")(decoder_input)

autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

In [ ]:
autoencoder.compile(optimizer = 'adam',
                    loss = 'mae',
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 5)